In [160]:
import imports; reload(imports)
from imports import *
import utils

In [2]:
path = '../data/'

In [3]:
train_df = pd.read_csv(path + 'train.csv')
valid_df = pd.read_csv(path + 'valid.csv')
test_df = pd.read_csv(path + 'test.csv')

In [ ]:
train_df

## Convert Functions

In [4]:
def sex_to_label(sex):
    return 0 if sex == 'male' else 1

In [5]:
def embarked_to_label(c):
    if (c == 'C'):
        return 0
    if (c == 'Q'):
        return 1
    return 2

In [112]:
def cabin_to_label(s):
    if not isinstance(s, str):
        return 0
    else:
        return ord(s[0]) - ord('A')

In [115]:
def transform_data(df):
    df = df.fillna(0)
    numerics_df = df[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']]
    sex_df = df['Sex'].apply(sex_to_label)
    cabin_df = df['Cabin'].apply(cabin_to_label)
    embarked_df = df['Embarked'].apply(embarked_to_label)
    data_df = pd.concat([numerics_df, sex_df, cabin_df, embarked_df], axis=1)
    return data_df

## Prepare Data

In [ ]:
sex_df = train_df['Sex'].apply(sex_to_label)

In [ ]:
embarked_df = train_df['Embarked'].apply(embarked_to_label)

In [113]:
cabin_df = train_df['Cabin'].apply(cabin_to_label)

In [ ]:
type(train_df['Cabin'][1]) == type('str')
isinstance(train_df['Cabin'][0], str)

In [ ]:
train_df['Cabin'].apply(cabin_to_label)

In [ ]:
pd.concat([sex_df, embarked_df], axis=1)

In [ ]:
train_df[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']]

In [ ]:
transform_data(train_df[:10])

In [116]:
train = np.array(transform_data(train_df))
train_label = to_categorical(np.array(train_df['Survived']))

In [117]:
valid = np.array(transform_data(valid_df))
valid_label = to_categorical(np.array(valid_df['Survived']))

In [118]:
test = np.array(transform_data(test_df))

In [119]:
train.shape[1]
valid.shape

(91, 8)

## Create Model

In [321]:
model = Sequential([
#     Dense(input_dim=train.shape[1], output_dim=2, activation='softmax')
    Dense(input_dim=train.shape[1], output_dim=8, activation='relu'),
    Dropout(0.5),
    Dense(output_dim=2, activation='softmax')
])
model.compile(optimizer=Adam(lr=1E-3), metrics=['accuracy'], loss='categorical_crossentropy')

In [235]:
model.optimizer.lr.get_value()

array(9.999999747378752e-05, dtype=float32)

In [269]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_11 (Dense)                 (None, 14)            126         dense_input_6[0][0]              
____________________________________________________________________________________________________
dropout_6 (Dropout)              (None, 14)            0           dense_11[0][0]                   
____________________________________________________________________________________________________
dense_12 (Dense)                 (None, 2)             30          dropout_6[0][0]                  
Total params: 156
____________________________________________________________________________________________________


In [297]:
model.optimizer.lr = 1E-5

In [360]:
model.fit(train, train_label, validation_data=(valid, valid_label), batch_size=16, nb_epoch=10)

Train on 800 samples, validate on 91 samples
Epoch 1/10
800/800 [==============================] - 0s - loss: 0.5284 - acc: 0.7188 - val_loss: 0.3968 - val_acc: 0.8022
Epoch 2/10
800/800 [==============================] - 0s - loss: 0.5169 - acc: 0.7200 - val_loss: 0.4099 - val_acc: 0.7912
Epoch 3/10
800/800 [==============================] - 0s - loss: 0.5086 - acc: 0.7262 - val_loss: 0.4074 - val_acc: 0.7912
Epoch 4/10
800/800 [==============================] - 0s - loss: 0.5071 - acc: 0.7200 - val_loss: 0.4101 - val_acc: 0.7912
Epoch 5/10
800/800 [==============================] - 0s - loss: 0.5124 - acc: 0.7212 - val_loss: 0.4041 - val_acc: 0.7912
Epoch 6/10
800/800 [==============================] - 0s - loss: 0.5201 - acc: 0.7212 - val_loss: 0.4193 - val_acc: 0.7802
Epoch 7/10
800/800 [==============================] - 0s - loss: 0.5221 - acc: 0.7163 - val_loss: 0.4200 - val_acc: 0.7912
Epoch 8/10
800/800 [==============================] - 0s - loss: 0.5222 - acc: 0.7113 - val_lo

In [316]:
pred = model.predict(test, batch_size=16)

In [317]:
pred[:10]

array([[ 0.85069931,  0.14930066],
       [ 0.61502182,  0.38497815],
       [ 0.85175037,  0.14824966],
       [ 0.86343932,  0.13656068],
       [ 0.56466502,  0.43533498],
       [ 0.84470129,  0.15529868],
       [ 0.44521356,  0.55478644],
       [ 0.73705238,  0.26294765],
       [ 0.32059988,  0.67940015],
       [ 0.89771992,  0.10228006]], dtype=float32)

In [318]:
res = np.argmax(pred, axis=1)

In [319]:
res[:10]

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0])

## Submit

In [320]:
res_df = pd.concat([test_df['PassengerId'], pd.DataFrame(res)], axis=1)
res_df.columns = ['PassengerId', 'Survived']
res_df.to_csv(path + '../result/result.csv', index=False)